# Train-test split strategy

In [5]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"phuongkhanh21","key":"0fd6eb70e1509aad441adeb76f0dfff2"}'}

In [6]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d berkanoztas/synthetic-transaction-monitoring-dataset-aml

import zipfile

with zipfile.ZipFile("synthetic-transaction-monitoring-dataset-aml.zip", 'r') as zip_ref:
    zip_ref.extractall("synthetic_transaction_data")

import pandas as pd
os.listdir("synthetic_transaction_data")
df = pd.read_csv("synthetic_transaction_data/SAML-D.csv")
df.head(5)

Dataset URL: https://www.kaggle.com/datasets/berkanoztas/synthetic-transaction-monitoring-dataset-aml
License(s): CC-BY-NC-SA-4.0
synthetic-transaction-monitoring-dataset-aml.zip: Skipping, found more recently modified local copy (use --force to force download)


,Time,Date,Sender_account,Receiver_account,Amount,Payment_currency,Received_currency,Sender_bank_location,Receiver_bank_location,Payment_type,Is_laundering,Laundering_type
0,10:35:19,2022-10-07,8724731955,2769355426,1459.15,UK pounds,UK pounds,UK,UK,Cash Deposit,0,Normal_Cash_Deposits
1,10:35:20,2022-10-07,1491989064,8401255335,6019.64,UK pounds,Dirham,UK,UAE,Cross-border,0,Normal_Fan_Out
2,10:35:20,2022-10-07,287305149,4404767002,14328.44,UK pounds,UK pounds,UK,UK,Cheque,0,Normal_Small_Fan_Out
3,10:35:21,2022-10-07,5376652437,9600420220,11895.00,UK pounds,UK pounds,UK,UK,ACH,0,Normal_Fan_In
4,10:35:21,2022-10-07,9614186178,3803336972,115.25,UK pounds,UK pounds,UK,UK,Cash Deposit,0,Normal_Cash_Deposits


In [7]:
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_timedelta(df['Time'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9504852 entries, 0 to 9504851
Data columns (total 12 columns):
 #   Column                  Dtype          
---  ------                  -----          
 0   Time                    timedelta64[ns]
 1   Date                    datetime64[ns] 
 2   Sender_account          int64          
 3   Receiver_account        int64          
 4   Amount                  float64        
 5   Payment_currency        object         
 6   Received_currency       object         
 7   Sender_bank_location    object         
 8   Receiver_bank_location  object         
 9   Payment_type            object         
 10  Is_laundering           int64          
 11  Laundering_type         object         
dtypes: datetime64[ns](1), float64(1), int64(3), object(6), timedelta64[ns](1)
memory usage: 870.2+ MB


In [8]:
def sample_data(df, validation_dt=70, test_dt=40):
  # validation_cut_off=70: selects transactions for validation set, from the final 70 days prior to the dataset's latest date
  # test_cut_off=40: selects transactions for test set, from the final 70 days prior to the dataset's latest date

  test_cutoff = df['Date'].max() - pd.Timedelta(days=test_dt)
  validation_cutoff = df['Date'].max() - pd.Timedelta(days=validation_dt)

  test_size = len(df[df.Date >= test_cutoff])
  validation_size = len(df[(df.Date >= validation_cutoff) & (df.Date < test_cutoff)])
  train_size = len(df[df.Date < validation_cutoff])

  test_percentage = test_size/len(df)
  validation_percentage = validation_size/len(df)
  train_percentage = train_size/len(df)

  print(f"Test size: {test_size} ({test_percentage*100:.2f}%)")
  print(f"Validation size: {validation_size} ({validation_percentage*100:.2f}%)")
  print(f"Train size: {train_size} ({train_percentage*100:.2f}%)")

  test_laundering = df[df.Date >= test_cutoff].Is_laundering.sum()/len(df[df.Date >= test_cutoff])
  validation_laundering = df[(df.Date >= validation_cutoff) & (df.Date < test_cutoff)].Is_laundering.sum()/len(df[(df.Date >= validation_cutoff) & (df.Date < test_cutoff)])
  train_laundering = df[df.Date < validation_cutoff].Is_laundering.sum()/len(df[df.Date < validation_cutoff])

  print(f"Laundering proportion (Test set): {test_laundering}")
  print(f"Laundering proportion (Validation set): {validation_laundering}")
  print(f"Laundering proportion (Train set): {train_laundering}")

In [9]:
sample_data(df, 70, 40)

Test size: 1177539 (12.39%)
Validation size: 897694 (9.44%)
Train size: 7429619 (78.17%)
Laundering proportion (Test set): 0.0011693880202693924
Laundering proportion (Validation set): 0.0011329027485980747
Laundering proportion (Train set): 0.0010066465050226666


In [10]:
sample_data(df, 80, 40)

Test size: 1177539 (12.39%)
Validation size: 1195050 (12.57%)
Train size: 7132263 (75.04%)
Laundering proportion (Test set): 0.0011693880202693924
Laundering proportion (Validation set): 0.0011606208945232417
Laundering proportion (Train set): 0.0009967383423746432


In [12]:
sample_data(df, 70, 35)

Test size: 1030388 (10.84%)
Validation size: 1044845 (10.99%)
Train size: 7429619 (78.17%)
Laundering proportion (Test set): 0.0011626688199008529
Laundering proportion (Validation set): 0.0011446673908570171
Laundering proportion (Train set): 0.0010066465050226666
